# 🧹 TIỀN XỬ LÝ DỮ LIỆU ĐỘT QUỴ - HEALTHCARE

## 🔍 Giới thiệu

Notebook này thực hiện các bước tiền xử lý dữ liệu cho bộ dữ liệu về đột quỵ, tiếp nối từ phân tích khám phá dữ liệu (EDA) đã thực hiện trong notebook trước. Mục tiêu là chuẩn bị dữ liệu sạch và có cấu trúc tốt cho việc xây dựng mô hình dự đoán đột quỵ.

### 📋 Mục tiêu tiền xử lý

- Xử lý các giá trị thiếu (null values), đặc biệt là trong cột BMI
- Xử lý các giá trị ngoại lai (outliers) trong các biến số học
- Mã hóa các biến phân loại (categorical variables)
- Chuẩn hóa các biến số học (numerical variables)
- Tạo các biến mới có ý nghĩa (feature engineering)
- Cân bằng dữ liệu cho biến mục tiêu (đột quỵ)

### 🔄 Quy trình tiền xử lý

1. **Tải dữ liệu**: Tải dữ liệu từ kết quả của notebook phân tích khám phá
2. **Xử lý giá trị thiếu**: Áp dụng các phương pháp phù hợp để xử lý giá trị thiếu
3. **Xử lý giá trị ngoại lai**: Phát hiện và xử lý các giá trị ngoại lai
4. **Biến đổi dữ liệu**: Mã hóa biến phân loại và chuẩn hóa biến số học
5. **Tạo biến mới**: Tạo các đặc trưng mới có ý nghĩa
6. **Cân bằng dữ liệu**: Xử lý vấn đề mất cân bằng trong biến mục tiêu
7. **Lưu dữ liệu đã xử lý**: Lưu dữ liệu đã tiền xử lý để sử dụng cho việc xây dựng mô hình

## 📚 Import các thư viện cần thiết

Để thực hiện tiền xử lý dữ liệu, chúng ta cần import các thư viện Python phổ biến cho xử lý và biến đổi dữ liệu.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Thư viện cho xử lý dữ liệu
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split

# Thư viện cho cân bằng dữ liệu
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette('viridis')

# Bỏ qua cảnh báo
import warnings
warnings.filterwarnings('ignore')

# Hiển thị tất cả các cột
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'sklearn'

## 📥 Tải dữ liệu

Tải dữ liệu từ file CSV hoặc từ kết quả của notebook phân tích khám phá.

In [ ]:
df = pd.read_csv('/Users/quanglong/PHÂN TÍCH DỮ LIỆU /data/raw/healthcare-dataset-stroke-data.csv.xls')
df.head()


## 🔍 Kiểm tra dữ liệu

Kiểm tra thông tin cơ bản của dữ liệu trước khi tiến hành xử lý.

In [ ]:
# Kiểm tra thông tin cơ bản
df.info()

# Kiểm tra giá trị thiếu
print('
Số lượng giá trị thiếu trong mỗi cột:')
print(df.isnull().sum())

# Kiểm tra thống kê mô tả
print('
Thống kê mô tả cho các biến số học:')
print(df.describe())

# Kiểm tra phân phối của biến mục tiêu
print('
Phân phối của biến mục tiêu (stroke):')
print(df['stroke'].value_counts())
print(f'Tỷ lệ đột quỵ: {df["stroke"].mean()*100:.2f}%')

## 🧹 Xử lý giá trị thiếu (Missing Values)

Từ phân tích khám phá dữ liệu, chúng ta đã biết rằng cột BMI có giá trị thiếu. Chúng ta sẽ xử lý các giá trị thiếu này bằng các phương pháp phù hợp.

In [ ]:
# Trực quan hóa phân phối của BMI để chọn phương pháp điền giá trị thiếu phù hợp
plt.figure(figsize=(10, 6))
sns.histplot(df['bmi'].dropna(), kde=True)
plt.title('Phân phối của BMI')
plt.xlabel('BMI')
plt.ylabel('Số lượng')
plt.show()

# Phương pháp 1: Điền giá trị thiếu bằng trung vị (median)
df_median = df.copy()
df_median['bmi'].fillna(df['bmi'].median(), inplace=True)

# Phương pháp 2: Điền giá trị thiếu bằng KNN Imputer
df_knn = df.copy()
# Chọn các biến số học để sử dụng trong KNN Imputer
numeric_features = ['age', 'avg_glucose_level', 'bmi']
# Khởi tạo KNN Imputer
imputer = KNNImputer(n_neighbors=5)
# Áp dụng KNN Imputer
df_knn[numeric_features] = imputer.fit_transform(df_knn[numeric_features])

# So sánh phân phối sau khi điền giá trị thiếu
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(df['bmi'].dropna(), kde=True, color='blue')
plt.title('BMI - Dữ liệu gốc (không có giá trị thiếu)')
plt.xlabel('BMI')
plt.ylabel('Số lượng')

plt.subplot(1, 3, 2)
sns.histplot(df_median['bmi'], kde=True, color='green')
plt.title('BMI - Điền bằng trung vị')
plt.xlabel('BMI')
plt.ylabel('Số lượng')

plt.subplot(1, 3, 3)
sns.histplot(df_knn['bmi'], kde=True, color='red')
plt.title('BMI - Điền bằng KNN')
plt.xlabel('BMI')
plt.ylabel('Số lượng')

plt.tight_layout()
plt.show()

# Chọn phương pháp điền giá trị thiếu tốt nhất dựa trên kết quả so sánh
# Trong trường hợp này, chúng ta sẽ sử dụng KNN Imputer vì nó bảo toàn phân phối tốt hơn
df = df_knn.copy()

# Kiểm tra lại giá trị thiếu sau khi xử lý
print('Số lượng giá trị thiếu sau khi xử lý:')
print(df.isnull().sum())

## 📊 Xử lý giá trị ngoại lai (Outliers)

Phát hiện và xử lý các giá trị ngoại lai trong các biến số học như tuổi, mức đường huyết trung bình và BMI.

In [ ]:
# Trực quan hóa phân phối của các biến số học để phát hiện giá trị ngoại lai
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(x=df['age'])
plt.title('Biểu đồ Box Plot của Tuổi')
plt.xlabel('Tuổi')

plt.subplot(1, 3, 2)
sns.boxplot(x=df['avg_glucose_level'])
plt.title('Biểu đồ Box Plot của Mức đường huyết trung bình')
plt.xlabel('Mức đường huyết trung bình')

plt.subplot(1, 3, 3)
sns.boxplot(x=df['bmi'])
plt.title('Biểu đồ Box Plot của BMI')
plt.xlabel('BMI')

plt.tight_layout()
plt.show()

# Phương pháp 1: Cắt bỏ giá trị ngoại lai bằng IQR
def remove_outliers_iqr(df, column, lower_quantile=0.25, upper_quantile=0.75, whisker_width=1.5):
    # Tính IQR
    Q1 = df[column].quantile(lower_quantile)
    Q3 = df[column].quantile(upper_quantile)
    IQR = Q3 - Q1
    
    # Xác định ngưỡng
    lower_bound = Q1 - whisker_width * IQR
    upper_bound = Q3 + whisker_width * IQR
    
    # Lọc dữ liệu
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Phương pháp 2: Giới hạn giá trị ngoại lai (capping)
def cap_outliers_iqr(df, column, lower_quantile=0.25, upper_quantile=0.75, whisker_width=1.5):
    # Tính IQR
    Q1 = df[column].quantile(lower_quantile)
    Q3 = df[column].quantile(upper_quantile)
    IQR = Q3 - Q1
    
    # Xác định ngưỡng
    lower_bound = Q1 - whisker_width * IQR
    upper_bound = Q3 + whisker_width * IQR
    
    # Giới hạn giá trị
    df_capped = df.copy()
    df_capped[column] = np.where(df_capped[column] < lower_bound, lower_bound, df_capped[column])
    df_capped[column] = np.where(df_capped[column] > upper_bound, upper_bound, df_capped[column])
    
    return df_capped

# Áp dụng phương pháp giới hạn giá trị ngoại lai cho các biến số học
df_capped = df.copy()
for column in ['avg_glucose_level', 'bmi']:
    df_capped = cap_outliers_iqr(df_capped, column)

# Lưu ý: Chúng ta không xử lý giá trị ngoại lai cho biến tuổi vì tuổi là một biến quan trọng
# và các giá trị cao có thể là hợp lệ trong ngữ cảnh của đột quỵ

# So sánh phân phối trước và sau khi xử lý giá trị ngoại lai
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.boxplot(x=df['avg_glucose_level'])
plt.title('Mức đường huyết trung bình - Trước khi xử lý')
plt.xlabel('Mức đường huyết trung bình')

plt.subplot(2, 2, 2)
sns.boxplot(x=df_capped['avg_glucose_level'])
plt.title('Mức đường huyết trung bình - Sau khi xử lý')
plt.xlabel('Mức đường huyết trung bình')

plt.subplot(2, 2, 3)
sns.boxplot(x=df['bmi'])
plt.title('BMI - Trước khi xử lý')
plt.xlabel('BMI')

plt.subplot(2, 2, 4)
sns.boxplot(x=df_capped['bmi'])
plt.title('BMI - Sau khi xử lý')
plt.xlabel('BMI')

plt.tight_layout()
plt.show()

# Sử dụng dữ liệu đã xử lý giá trị ngoại lai
df = df_capped.copy()

## 🔄 Biến đổi dữ liệu (Data Transformation)

Thực hiện các biến đổi dữ liệu như mã hóa biến phân loại và chuẩn hóa biến số học.

In [ ]:
# Mã hóa biến phân loại
# Sử dụng One-Hot Encoding cho các biến phân loại có nhiều giá trị
categorical_features = ['gender', 'work_type', 'Residence_type', 'smoking_status']

# Tạo bản sao của dữ liệu
df_encoded = df.copy()

# Mã hóa biến nhị phân
df_encoded['ever_married'] = df_encoded['ever_married'].map({'Yes': 1, 'No': 0})

# Mã hóa One-Hot cho các biến phân loại khác
df_encoded = pd.get_dummies(df_encoded, columns=categorical_features, drop_first=True)

# Chuẩn hóa biến số học
numeric_features = ['age', 'avg_glucose_level', 'bmi']

# Phương pháp 1: StandardScaler (Z-score normalization)
scaler = StandardScaler()
df_encoded[numeric_features] = scaler.fit_transform(df_encoded[numeric_features])

# Hiển thị dữ liệu sau khi biến đổi
print('Dữ liệu sau khi biến đổi:')
print(df_encoded.head())
print('các cột sau khi biến đổi:')
print(df_encoded.columns.tolist())

## 🔧 Tạo biến mới (Feature Engineering)

Tạo các biến mới có ý nghĩa để cải thiện hiệu suất của mô hình.

In [ ]:
# Tạo bản sao của dữ liệu đã biến đổi
df_featured = df_encoded.copy()

# Tạo biến mới: Nhóm tuổi
def age_group(age):
    if age < 0: # Chuẩn hóa có thể tạo ra giá trị âm
        return 0  # Trẻ em
    elif age < 1: # Sau chuẩn hóa, giá trị từ 0-18 có thể nằm trong khoảng này
        return 1  # Thanh thiếu niên
    elif age < 2: # Sau chuẩn hóa, giá trị từ 18-40 có thể nằm trong khoảng này
        return 2  # Người trưởng thành
    elif age < 3: # Sau chuẩn hóa, giá trị từ 40-65 có thể nằm trong khoảng này
        return 3  # Trung niên
    else:        # Sau chuẩn hóa, giá trị > 65 có thể nằm trong khoảng này
        return 4  # Người cao tuổi

df_featured['age_group'] = df_featured['age'].apply(age_group)

# Tạo biến mới: Phân loại BMI theo WHO
def bmi_category(bmi):
    # Lưu ý: BMI đã được chuẩn hóa, nên chúng ta cần điều chỉnh ngưỡng
    # Đây chỉ là ví dụ, trong thực tế cần tính toán lại ngưỡng sau chuẩn hóa
    if bmi < -1:  # Tương đương BMI < 18.5 trước chuẩn hóa
        return 0  # Thiếu cân
    elif bmi < 0:  # Tương đương 18.5 <= BMI < 25 trước chuẩn hóa
        return 1  # Bình thường
    elif bmi < 1:  # Tương đương 25 <= BMI < 30 trước chuẩn hóa
        return 2  # Thừa cân
    else:         # Tương đương BMI >= 30 trước chuẩn hóa
        return 3  # Béo phì

df_featured['bmi_category'] = df_featured['bmi'].apply(bmi_category)

# Tạo biến mới: Phân loại mức đường huyết
def glucose_category(glucose):
    # Lưu ý: Mức đường huyết đã được chuẩn hóa, nên chúng ta cần điều chỉnh ngưỡng
    # Đây chỉ là ví dụ, trong thực tế cần tính toán lại ngưỡng sau chuẩn hóa
    if glucose < -0.5:  # Tương đương glucose < 70 mg/dL trước chuẩn hóa
        return 0  # Thấp
    elif glucose < 0.5:  # Tương đương 70 <= glucose < 100 mg/dL trước chuẩn hóa
        return 1  # Bình thường
    elif glucose < 1.5:  # Tương đương 100 <= glucose < 126 mg/dL trước chuẩn hóa
        return 2  # Tiền tiểu đường
    else:              # Tương đương glucose >= 126 mg/dL trước chuẩn hóa
        return 3  # Tiểu đường

df_featured['glucose_category'] = df_featured['avg_glucose_level'].apply(glucose_category)

# Tạo biến mới: Chỉ số nguy cơ kết hợp (kết hợp các yếu tố nguy cơ)
df_featured['risk_score'] = df_featured['hypertension'] + df_featured['heart_disease'] + df_featured['bmi_category'] + df_featured['glucose_category']

# Hiển thị dữ liệu sau khi tạo biến mới
print('Dữ liệu sau khi tạo biến mới:')
print(df_featured.head())
print('Các cột sau khi tạo biến mới:')
print(df_featured.columns.tolist())

## ⚖️ Cân bằng dữ liệu (Data Balancing)

Xử lý vấn đề mất cân bằng trong biến mục tiêu (đột quỵ) để cải thiện hiệu suất của mô hình.

In [ ]:
# Kiểm tra phân phối của biến mục tiêu
print('Phân phối của biến mục tiêu (stroke):')
print(df_featured['stroke'].value_counts())
print(f'Tỷ lệ đột quỵ: {df_featured["stroke"].mean()*100:.2f}%')

# Trực quan hóa phân phối của biến mục tiêu
plt.figure(figsize=(8, 6))
sns.countplot(x=df_featured['stroke'])
plt.title('Phân phối của biến mục tiêu (stroke)')
plt.xlabel('Đột quỵ (0: Không, 1: Có)')
plt.ylabel('Số lượng')
plt.show()

# Chuẩn bị dữ liệu cho cân bằng
X = df_featured.drop('stroke', axis=1)
y = df_featured['stroke']

# Phương pháp 1: SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Phương pháp 2: Kết hợp Under-sampling và SMOTE
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=42)
X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(X, y)

# So sánh phân phối của biến mục tiêu sau khi cân bằng
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.countplot(x=y)
plt.title('Phân phối gốc')
plt.xlabel('Đột quỵ (0: Không, 1: Có)')
plt.ylabel('Số lượng')

plt.subplot(1, 3, 2)
sns.countplot(x=y_smote)
plt.title('Sau khi áp dụng SMOTE')
plt.xlabel('Đột quỵ (0: Không, 1: Có)')
plt.ylabel('Số lượng')

plt.subplot(1, 3, 3)
sns.countplot(x=y_smote_tomek)
plt.title('Sau khi áp dụng SMOTE-Tomek')
plt.xlabel('Đột quỵ (0: Không, 1: Có)')
plt.ylabel('Số lượng')

plt.tight_layout()
plt.show()

# Chọn phương pháp cân bằng dữ liệu tốt nhất
# Trong trường hợp này, chúng ta sẽ sử dụng SMOTE-Tomek vì nó kết hợp cả over-sampling và under-sampling
X_balanced = X_smote_tomek
y_balanced = y_smote_tomek

# Tạo DataFrame từ dữ liệu đã cân bằng
df_balanced = pd.DataFrame(X_balanced, columns=X.columns)
df_balanced['stroke'] = y_balanced

# Kiểm tra phân phối của biến mục tiêu sau khi cân bằng
print('
Phân phối của biến mục tiêu sau khi cân bằng:')
print(df_balanced['stroke'].value_counts())
print(f'Tỷ lệ đột quỵ: {df_balanced["stroke"].mean()*100:.2f}%')

## 🔄 Chia dữ liệu thành tập huấn luyện và tập kiểm tra

Chia dữ liệu đã xử lý thành tập huấn luyện và tập kiểm tra để chuẩn bị cho việc xây dựng mô hình.

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X = df_balanced.drop('stroke', axis=1)
y = df_balanced['stroke']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Kiểm tra kích thước của tập huấn luyện và tập kiểm tra
print('Kích thước của tập huấn luyện:', X_train.shape)
print('Kích thước của tập kiểm tra:', X_test.shape)

# Kiểm tra phân phối của biến mục tiêu trong tập huấn luyện và tập kiểm tra
print('
Phân phối của biến mục tiêu trong tập huấn luyện:')
print(y_train.value_counts())
print(f'Tỷ lệ đột quỵ trong tập huấn luyện: {y_train.mean()*100:.2f}%')

print('
Phân phối của biến mục tiêu trong tập kiểm tra:')
print(y_test.value_counts())
print(f'Tỷ lệ đột quỵ trong tập kiểm tra: {y_test.mean()*100:.2f}%')

## 💾 Lưu dữ liệu đã xử lý

Lưu dữ liệu đã tiền xử lý để sử dụng cho việc xây dựng mô hình.

In [ ]:
# Lưu dữ liệu đã xử lý
df_balanced.to_csv('/Users/quanglong/PHÂN TÍCH DỮ LIỆU /data/processed/stroke_processed.csv', index=False)

# Lưu tập huấn luyện và tập kiểm tra
X_train.to_csv('/Users/quanglong/PHÂN TÍCH DỮ LIỆU /data/processed/X_train.csv', index=False)
X_test.to_csv('/Users/quanglong/PHÂN TÍCH DỮ LIỆU /data/processed/X_test.csv', index=False)
y_train.to_csv('/Users/quanglong/PHÂN TÍCH DỮ LIỆU /data/processed/y_train.csv', index=False)
y_test.to_csv('/Users/quanglong/PHÂN TÍCH DỮ LIỆU /data/processed/y_test.csv', index=False)

print('Dữ liệu đã được lưu thành công!')

## 📝 Tổng kết

Trong notebook này, chúng ta đã thực hiện các bước tiền xử lý dữ liệu quan trọng để chuẩn bị cho việc xây dựng mô hình dự đoán đột quỵ:

1. **Xử lý giá trị thiếu**: Đã sử dụng KNN Imputer để điền giá trị thiếu trong cột BMI, giúp bảo toàn phân phối của dữ liệu.

2. **Xử lý giá trị ngoại lai**: Đã phát hiện và giới hạn các giá trị ngoại lai trong các biến số học như mức đường huyết trung bình và BMI.

3. **Biến đổi dữ liệu**: Đã mã hóa các biến phân loại và chuẩn hóa các biến số học để chuẩn bị cho các thuật toán machine learning.

4. **Tạo biến mới**: Đã tạo các biến mới có ý nghĩa như nhóm tuổi, phân loại BMI, phân loại mức đường huyết và chỉ số nguy cơ kết hợp.

5. **Cân bằng dữ liệu**: Đã xử lý vấn đề mất cân bằng trong biến mục tiêu bằng cách sử dụng kỹ thuật SMOTE-Tomek.

6. **Chia dữ liệu**: Đã chia dữ liệu thành tập huấn luyện và tập kiểm tra để chuẩn bị cho việc xây dựng mô hình.

Dữ liệu đã được tiền xử lý và lưu trữ để sử dụng trong notebook tiếp theo về xây dựng mô hình dự đoán đột quỵ.